# Neo4j MCP Agent with Strands Agents

Query a Neo4j graph database using **AWS Strands Agents** and **AgentCore Gateway MCP**.

Based on the [official AWS AgentCore sample](https://github.com/awslabs/amazon-bedrock-agentcore-samples/blob/main/01-tutorials/02-AgentCore-gateway/05-mcp-server-as-a-target/01-mcp-server-target.ipynb).

## 1. Setup

In [ ]:
%pip install strands-agents strands-agents-tools mcp httpx -q

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client

print("Imports OK")

## 2. Configuration

**Before running:** Get your credentials:
1. Run `./setup-inference-profile.sh haiku` and copy the ARN
2. Copy `GATEWAY_URL` and `ACCESS_TOKEN` from `.mcp-credentials.json`

In [ ]:
# Paste your values here
INFERENCE_PROFILE_ARN = "PASTE_YOUR_ARN_HERE"
GATEWAY_URL = "PASTE_YOUR_GATEWAY_URL_HERE"
ACCESS_TOKEN = "PASTE_YOUR_ACCESS_TOKEN_HERE"
REGION = "us-west-2"

# Validate
assert "PASTE" not in INFERENCE_PROFILE_ARN, "Set INFERENCE_PROFILE_ARN"
assert "PASTE" not in GATEWAY_URL, "Set GATEWAY_URL"
assert "PASTE" not in ACCESS_TOKEN, "Set ACCESS_TOKEN"
print(f"Region:  {REGION}")
print(f"Profile: {INFERENCE_PROFILE_ARN[:70]}...")
print(f"Gateway: {GATEWAY_URL[:50]}...")
print("Config OK")

## 3. Initialize Model & MCP Client

**Key pattern from AWS sample:** The transport factory returns a fresh `streamablehttp_client` each time, with the Bearer token embedded in headers.

In [ ]:
# Bedrock model
model = BedrockModel(
    model_id=INFERENCE_PROFILE_ARN,
    region_name=REGION,
    temperature=0,
)


# Token getter (called each time transport is created)
def get_token():
    return ACCESS_TOKEN


# Transport factory - returns fresh streamablehttp_client each call
def create_streamable_http_transport():
    return streamablehttp_client(
        GATEWAY_URL,
        headers={"Authorization": f"Bearer {get_token()}"}
    )


# MCP client with transport factory
mcp_client = MCPClient(create_streamable_http_transport)

print("Model and MCP client ready")

## 4. Test MCP Connection

In [ ]:
with mcp_client:
    tools = mcp_client.list_tools_sync()
    print(f"Connected! Found {len(tools)} tools:")
    for tool in tools:
        print(f"  - {tool.name}")

## 5. Create Agent & Query Function

In [ ]:
SYSTEM_PROMPT = """You are a Neo4j database assistant. You can:
- Get the database schema
- Run read-only Cypher queries

Always get the schema first, then query based on actual labels/relationships.
Be concise. Format results clearly."""


def query(question: str) -> str:
    """Query the Neo4j database via MCP."""
    print(f"Q: {question}")
    print("-" * 60)
    
    with mcp_client:
        tools = mcp_client.list_tools_sync()
        agent = Agent(
            model=model,
            tools=tools,
            system_prompt=SYSTEM_PROMPT,
        )
        result = agent(question)
    
    print(f"\nA: {result}")
    return str(result)

## 6. Demo Queries

In [ ]:
_ = query("What is the database schema?")

In [ ]:
_ = query("How many nodes are there by label?")

In [ ]:
_ = query("Show 5 sample records from the most populated node type.")

## 7. Your Query

In [ ]:
# _ = query("Your question here")